**Setup**

In [2]:
import pandas as pd

from abc import ABC, abstractmethod

class Transformation(ABC):
    @abstractmethod
    def transform(self) -> pd.DataFrame:
        pass

def filterDataFrame(data: pd.DataFrame, filters: dict = {}) -> pd.DataFrame:
    filteredData: pd.DataFrame = data

    for key in filters.keys():
        if filters[key] != None:
            filteredData = filteredData.loc[filteredData[key] == filters[key]]

    return filteredData

from enum import Enum

class ColName(Enum):
    SEX = "sex"
    UNIT = "unit"
    INSC_TYPE = "insc"
    OFFER = "offer"
    ID = "id"

import os

class StudentScholarshipsMovements(Transformation):
    def transform(
        self,
        table1: pd.DataFrame,
        table2: pd.DataFrame,
        scholarships: pd.DataFrame,
        filters1: object,
        filters2: object,
        schFilters: object,
        outputFolder: str,
    ) -> pd.DataFrame:

        table1 = filterDataFrame(table1, filters1)
        table2 = filterDataFrame(table2, filters2)
        scholarships = filterDataFrame(scholarships, schFilters)
        table1.to_excel(os.path.join(outputFolder , 'table1.xlsx'))
        table2.to_excel(os.path.join(outputFolder , 'table2.xlsx'))
        scholarships.to_excel(os.path.join(outputFolder , 'scholarships.xlsx'))

        activity: pd.DataFrame = table1.merge(
            table2, on=[ColName.ID.value], how="inner"
        )
        # change the names of the activity columns to avoid conflicts
        activity = activity.rename(
            columns={
                ColName.UNIT.value + "_x": "DX[" + ColName.UNIT.value + "]",
                ColName.OFFER.value + "_x": "DX[" + ColName.OFFER.value + "]",
                ColName.INSC_TYPE.value + "_x": "DX[" + ColName.INSC_TYPE.value + "]",
                ColName.SEX.value + "_x": "DX[" + ColName.SEX.value + "]",

                ColName.UNIT.value + "_y": "DXY[" + ColName.UNIT.value + "]",
                ColName.OFFER.value + "_y": "DXY[" + ColName.OFFER.value + "]",
                ColName.INSC_TYPE.value + "_y": "DXY[" + ColName.INSC_TYPE.value + "]",
                ColName.SEX.value + "_y": "DXY[" + ColName.SEX.value + "]",
            }
        )

        activity.to_excel(os.path.join(outputFolder , 'activity.xlsx'))

        # Conversion from categorical to string to allow comparison between series (only if columns are categorical)
        """
        activity[ColName.OFFER.value + "_x"] = activity[
            ColName.OFFER.value + "_x"
        ].astype(str)

        activity[ColName.OFFER.value + "_y"] = activity[
            ColName.OFFER.value + "_y"
        ].astype(str)
        """

        differentActivity: pd.DataFrame = activity.loc[
            activity["DX[" + ColName.OFFER.value + "]"] != activity["DXY[" + ColName.OFFER.value + "]"]
        ]

        differentActivity.to_excel(os.path.join(outputFolder , 'differentActivity.xlsx'))

        differentActivityWithScholarships = differentActivity.merge(
            scholarships,
            on=ColName.ID.value,
            how="inner",
        )

        differentActivityWithScholarships = differentActivityWithScholarships.rename(
            columns={
                ColName.UNIT.value : "DB[" + ColName.UNIT.value + "]",
                ColName.OFFER.value : "DB[" + ColName.OFFER.value + "]",
            }
        )

        differentActivityWithScholarships.to_excel(os.path.join(outputFolder , 'differentActivityWithScholarships.xlsx'))

        differentActivityWithScholarships: pd.DataFrame = (
            differentActivityWithScholarships.loc[
                differentActivityWithScholarships["DXY[" + ColName.OFFER.value + "]"]
                == differentActivityWithScholarships["DB[" + ColName.OFFER.value + "]"]
            ]
        )

        differentActivityWithScholarships.to_excel(os.path.join(outputFolder , 'differentActivityWithScholarships2.xlsx'))

        differentActivityWithScholarshipsUniques : pd.DataFrame = differentActivityWithScholarships.drop_duplicates(subset=[ColName.ID.value])

        differentActivityWithScholarshipsUniques.to_excel(os.path.join(outputFolder , 'differentActivityWithScholarshipsUniques.xlsx'))

        result : pd.DataFrame = (
            differentActivityWithScholarshipsUniques
            .groupby("DB[" + ColName.OFFER.value + "]")[
                "DB[" + ColName.OFFER.value + "]"
            ]  # if grouping by categorical column, set observed=True
            .count()
        )

        result.to_excel(os.path.join(outputFolder , 'result.xlsx'))

        return result


**EXECUTE**

In [17]:
transformer = StudentScholarshipsMovements();

tab1 = pd.read_pickle('./test_files/student_inscriptions_xxxx.pickle')
tab2 = pd.read_pickle('./test_files/student_inscriptions_yyyy.pickle')
sch = pd.read_pickle('./test_files/belgrano_yyyy.pickle')

#create folder for output
outputFolder = os.path.join("./test_output" , "scholarship_movements_test")
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

transformer.transform(tab1,tab2,sch,{},{},{}, outputFolder)

DB[offer]
e    2
f    1
h    1
Name: DB[offer], dtype: int64